In [181]:
import io
import json
import sqlite3 as sql
import pandas as pd
import os

def query(query_string, connection):
    return pd.read_sql_query(query_string, connection)

In [170]:
file = open('owl.sql', 'w+')

In [171]:
json_file = open('matches.json')
data = json.load(json_file)['content']
json_file.close()

In [172]:
match_schema = (
    'CREATE TABLE [MATCHES] ( '
    'MATCH_ID INT, '
    'TEAM_ONE_ID INT, '
    'TEAM_TWO_ID INT, '
    'TEAM_ONE_SCORE INT, '
    'TEAM_TWO_SCORE INT, '
    'WINNER_ID INT, '
    'TIMESTAMP DATETIME, '
    'PRIMARY KEY (MATCH_ID)'
    ');' 
)

In [173]:
file.write(match_schema + '\n\n')

174

In [174]:
for match in data:
    insert = (
        'INSERT INTO MATCHES '
        '(MATCH_ID, TEAM_ONE_ID, TEAM_TWO_ID, '
        'TEAM_ONE_SCORE, TEAM_TWO_SCORE, '
        'WINNER_ID, TIMESTAMP)'
        'VALUES (%d, %d, %d, %d, %d, %d, %d);'
        % (match['id'], match['competitors'][0]['id'],
          match['competitors'][1]['id'],
          match['scores'][0]['value'],
          match['scores'][1]['value'],
          match['winner']['id'],
          match['actualStartDate'])
    )
    file.write(insert+'\n')

In [175]:
#connection = sql.connect('test.db')

In [176]:
#cursor = connection.cursor()
#db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#database = db.read()
#cursor.executescript(database)

In [177]:
game_schema = (
    '\nCREATE TABLE [GAMES] ('
    'GAME_ID INT, '
    'MATCH_ID INT, '
    'MAP TEXT, '
    'PRIMARY KEY (GAME_ID)'
    ');\n\n'
)

file.write(game_schema)

85

In [178]:
for match in data:
    for game in match['games']:
        try:
            map_name = str(game['attributes']['map']).replace('-', ' ')
            insert = (
                'INSERT INTO GAMES (GAME_ID, MATCH_ID, MAP) VALUES '
                '(%d, %d, \'%s\');'
                % (game['id'], match['id'], map_name)
            )
            file.write(insert+'\n')
        except:
            print('Error')

Error
Error


In [185]:
#try:
#    connection = sql.connect('test.db')
#    cursor = connection.cursor()
#    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#    database = db.read()
#    cursor.executescript(database)
#except Exception as e:
#    print(e)
#    os.remove('test.db')

In [186]:
teams_schema = (
    '\nCREATE TABLE [TEAMS] ('
    'TEAM_ID INT, '
    'TEAM_NAME TEXT, '
    'TEAM_LOCATION TEXT, '
    'PRIMARY KEY (TEAM_ID)'
    ');\n\n'
)

file.write(teams_schema)

97

In [190]:
team_list = []
for match in data:
    for competitor in match['competitors']:
        if competitor['id'] not in team_list:
            team_list.append(competitor['id'])
            insert = (
                    'INSERT INTO TEAMS (TEAM_ID, TEAM_NAME, TEAM_LOCATION) VALUES '
                    '(%d, \'%s\', \'%s\');'
                    % (competitor['id'], 
                       competitor['name'], 
                       competitor['homeLocation'])
                )
            file.write(insert+'\n')

In [ ]:
try:
    connection = sql.connect('test.db')
    cursor = connection.cursor()
    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
    database = db.read()
    cursor.executescript(database)
except Exception as e:
    print(e)
    os.remove('test.db')